# Plume model by Lazeroms _et al._ (2020)

In [ ]:
import sys
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("..")
matplotlib.rc_file('../rc_file')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

from Plume import PlumeModel
from ideal_geometry import IdealGeometry

# 1D use

## reference case

In [ ]:
dp = IdealGeometry('plumeref').geometry()
ds = PlumeModel(dp).compute_plume(full_nondim=True)

In [ ]:
ds

<img src="../../doc/Lazeroms2019/FigA1.png" width="400">

In [ ]:
plt.figure()
plt.axhline(c='k', lw=.5)
ds.M_full.plot(label='melt rate', c='lime')
ds.phi0_full.plot(label=r'circulation $\phi_0$', c='k')
plt.ylabel('non-dimensional quantities')
plt.xlim((0,1))
plt.ylim((-1,1))
plt.legend()
plt.savefig('../../results/Lazeroms2019/FigA1')

## Fig. 4: varying geometries, $T_a=-1.9^\circ C, S_a = 34.65 psu$ 
<img src="../../doc/Lazeroms2019/Fig4a.png" width="300">  <img src="../../doc/Lazeroms2019/Fig4b.png" width="300">

In [ ]:
f, ax = plt.subplots(1,2, figsize=(8,3))
for i in range(2):
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].set_xlabel([r'$X$ [km]',r'$\tilde{x}$'][i])
    ax[i].set_ylabel([r'$\dot{m}$  [m yr$^{-1}$]',r'$\mathcal{M}_0$'][i])
for i, slope in enumerate([.002,.003,.001,.002,.002]):
    zgl = [-1000,-1000,-1000,-500,-2000][i]
    pdict = {'x':np.linspace(0,abs(zgl/slope),N)}  # [m]
    c = ['k','b','cyan','r','lime'][i]
    pdict['draft'] = np.linspace(zgl,0,N)  # [m]
    pdict['Ta'], pdict['Sa'] = -1.9, 34.65
    dp = IdealGeometry('plume1D', pdict).geometry()
    ds = PlumeModel(dp).compute_plume(full_nondim=True)
    
    ax[0].plot(ds.x/1e3, ds.m, c=c)
    if i==0:
        ax[1].plot(ds.x_, ds.M_full, c='k')
ax[0].set_ylim((-5,4))
ax[0].set_xlim((0,1000))
ax[1].set_xlim((0,1))
ax[1].set_ylim((-.4,.4))
plt.savefig('../../results/Lazeroms2019/Fig4')

## Fig. 6: varying uniform ambient temperaturs $T_a$
<img src="../../doc/Lazeroms2019/Fig6.png" width="600">

In [ ]:
# Fig.6 parameters
pdict = dict(x=np.linspace(0,5e5,N), draft=np.linspace(-1000,0,N), Sa=34.65)  # [m]
f, ax = plt.subplots(1, 3, figsize=(10,3))
for i in range(3):
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].set_xlabel([r'$X$ [km]',r'$\tilde{x}$',r'$X$ [km]'][i])
    ax[i].set_ylabel([r'$\dot{m}$  [m yr$^{-1}$]',r'$\mathcal{M}_0$',r'$z_b$  [m]'][i])
cmap = matplotlib.cm.get_cmap('magma')(np.linspace(0.3,.8,6))
for i, Ta in enumerate([-1.9,-1.1,-.3,.5,1.3,2.1]):  # [degC]
    c = cmap[i]
    pdict['Ta'] = Ta
    dp = IdealGeometry('plume1D', pdict).geometry()
    ds = PlumeModel(dp).compute_plume(full_nondim=True)
    ax[0].plot(ds.x/1e3, ds.m, c=c)
    if i==0:
        ax[1].plot(ds.x_, ds.M_full, c='k')
        ax[2].plot(ds.x/1e3, ds.draft)
    ax[1].plot(ds.dgrl_, ds.M, c=c)
    ax[1].scatter(ds.dgrl_[-1], ds.M[-1], color=c, marker='o')
    plt.savefig('../../results/Lazeroms2019/Fig6')

# 2D use

In [ ]:
def create_2D_quasi_1D_plume_dataset(dp):
        """ adds 2nd spatial dimension, to test 2D capabilities """
        dp = dp.expand_dims({'y':np.arange(2)})
        return dp

dp = IdealGeometry('plumeref').geometry()
ds = PlumeModel(dp).compute_plume(full_nondim=True)

dp2 = create_2D_quasi_1D_plume_dataset(dp)
ds2 = PlumeModel(dp2).compute_plume(full_nondim=True)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,3))
ds.M.plot(ax=ax[0])
ds2.M.mean('y').plot(ax=ax[0])

ds.m.plot(ax=ax[1])
ds2.m.mean('y').plot(ax=ax[1])